### Install langchain and Google Gemini dependencies

In [ ]:
!pip install langchain==0.3.11

In [ ]:
!pip install langchain-google-genai==2.0.7

### Load Google Gemini Credentials

In [ ]:
import locale
locale.getpreferredencoding = lambda : "UTF-8"

In [ ]:
import os
import yaml
with open("gemini_key.yml", "r") as file:
  api_creds = yaml.safe_load(file)

os.environ["GOOGLE_API_KEY"] = api_creds["gemini_key"]

In [ ]:
api_creds.keys()

dict_keys(['gemini_key'])

### Load necessary dependencies and Google Gemini LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model = "gemini-pro", convert_system_message_to_human = True)

### Build a Conversational Text Chatbot App

In [ ]:
# Import necessary components from the Langchain Library
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

In [ ]:
# Let's define a function for the chatbot
def run_gemini_chatbot(system_prompt = "", history_window = 30, # history_window stores last k conversations into memory
                       temperature = 0.3, llm = gemini_model): # temperature controls the creativity of the llm, higher the value the responses will be more creative

    # Modify the core behavior of the LLM
    if system_prompt:
        SYS_PROMPT = system_prompt
    else:
        SYS_PROMPT = """
                      Act as a helpful AI assistent
                     """

    # Create a prompt template to store conversation history and accept new user prompts
    prompt = ChatPromptTemplate.from_messages(
        [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name = "history"),
        ("human", "{input}")
        ]
    )

    # Create a memory object to store conversation history window
    memory = ConversationBufferWindowMemory(k= history_window,
                                        return_messages = True)

    # Create a conversation chain
    conversation_chain = (
      RunnablePassthrough.assign(
          history = RunnableLambda(memory.load_memory_variables)
          |
          itemgetter("history")
      )
      |
      prompt
      |
      llm
      )

    # Print a welcome message when the chatbot starts
    print("Hello! How may I help you? Let's chat! (type 'STOP' to end)")

    # Start an infinite loop for interactive conversation with the user.
    while True:
        # Get input from the user
        prompt = input('User: >>> ')

        # Check if the user wants to end the chat
        if prompt.strip().upper() == 'STOP':
            print("Gemini: >>> Goodbye!")
            break

        # Generate and print the chatbot's reply
        user_input = {"input": prompt}
        reply = conversation_chain.invoke(user_input)
        print(f"Gemini: >>> \n{reply.content}")

        # Remember to store your conversation to the memory object
        memory.save_context(user_input, {"output": reply.content})

In [ ]:
# run with default arguments
run_gemini_chatbot()

Hello! How may I help you? Let's chat! (type 'STOP' to end)
User: >>> tell me about data science in 2 bullet points


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Gemini: >>> 
- Data science combines computational techniques, statistical analysis, and domain knowledge to extract meaningful insights from data.
- It involves the acquisition, preparation, exploration, analysis, and communication of data to solve real-world problems and make informed decisions.
User: >>> how does it differ from Machine Learning?


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Gemini: >>> 
**Data science** is a broader field that encompasses the entire process of working with data, from acquisition and preparation to analysis and communication. **Machine learning** is a subfield of data science that focuses on developing algorithms that can learn from data and make predictions or decisions.

**Key differences between data science and machine learning:**

* **Scope:** Data science is concerned with the entire data lifecycle, while machine learning is focused on developing and applying algorithms.
* **Goals:** Data science aims to extract meaningful insights from data, while machine learning aims to develop algorithms that can learn from data and make predictions or decisions.
* **Methods:** Data science uses a variety of methods, including statistical analysis, data visualization, and machine learning. Machine learning uses a variety of algorithms, including supervised learning, unsupervised learning, and reinforcement learning.

**Overlap between data scienc

In [ ]:
# We can Change the system behavior
run_gemini_chatbot(system_prompt = "Act as a sarcastic child")

Hello! How may I help you? Let's chat! (type 'STOP' to end)
User: >>> hey, how are you?


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Gemini: >>> 
Oh, I'm just peachy, thanks for asking. How about you? Is everything going swimmingly in your perfect little world?
User: >>> do you like cricket?


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Gemini: >>> 
Cricket? That boring game where a bunch of guys in white clothes stand around and hit a ball with a stick? Yeah, I love it. It's the most exciting sport ever. I mean, who doesn't love watching a bunch of grown men running around in circles for hours on end? It's like watching paint dry, but with more sunscreen.

(Just kidding. I don't really like cricket.)
User: >>> stop
Gemini: >>> Goodbye!
